In [46]:
import pandas as pd
import numpy as np
import statistics
import csv
import numbers

Будущих бухгалтеров отталкивает сложность современных электронных таблиц. Разработайте для них программу с минимальным функционалом: хранение числовых и текстовых данных в таблице с изменяемым размером.
1. Ячейка электронной таблицы: числовое или текстовое значение, может быть пустой
2. Ячейка с формулой: диапазон ячеек (адреса первой и последней ячейки) и операция над диапазоном (сумма, произведение, среднее значение), метод вывода результата операции (или ошибки, если расчёт невозможен).
3. Таблица: хранение ячеек и вычисления операций над ними.


In [139]:
class SimpleTable:
    """
    Таблица читается из csv файла
    
    Note:
        Дефолтная кодировка utf-8-sig
    
    Attributes
    ----------
    file : str
        путь до текстового файла
    encoding : str
        кодировка исходного файла с таблицей

    Methods
    -------
    print_table
        печатает таблицу
    cell(row, col)
        возвращает значения в ячейке
        rows-1 и col-1 -- чтобы адрес ячейки можно было задавать не 
        в питоновском формате, а в человеческом (значения начинаются с 1)
    cell_change(row, col, value)
        изменить значение ячейки
    add_row(row_pos=-1, row_values=[]) 
        добавить строку;
        если параметры не указаны, в самый низ таблицы добавляется строка
        с пустыми значениями (None)
    add_col(col_pos=-1, col_values=[])
        добавить столбец;
        если параметры не указаны, добавляется крайний правый столбец
        с пустыми значениями (None)
    delete_row(row_pos)
        удалить строку
    delete_col(col_pos)
        удалить столбец
    set_range(row1, col1, row2, col2)
        возвращает массив значений из заданного диапазона;
        диапазон может быть выбран только внутри одного столбца или одной строки
    range_operation(row1, col1, row2, col2, operation='sum')
        операции над диапазоном;
        operation='sum': сумма
        operation='mult': произведение
        operation='mean': среднее
    cell_operation(row1, col1, row2, col2, operation='add')
        операции над ячейками;
        operation='add: сумма (для числовых значений) или конкатенация строк (для строк)
        operation='replace': меняет местами значения в ячейках
        operation='min': наименьшее число или более короткая из строк
        operation='max': наибольшее число или более длинная из строк
        
    """
    def __init__(self, file, encoding = 'utf-8-sig'):
        self.table = []
        with open(file, encoding=encoding, newline='') as File:  
            reader = csv.reader(File, delimiter=';')
            for row in reader:
                for i in range(0,len(row)):
                    if row[i] == '':
                        row[i] = None
                    else:
                        try:
                            row[i] = int(row[i])
                        except:
                            row[i] = row[i]
                self.table.append(row)
    
    def print_table(self):
        mapper = lambda x: x+1
        df = pd.DataFrame(data=self.table)
        df.rename(mapper=mapper, axis=1, inplace=True)
        df.rename(mapper=mapper, axis=0, inplace=True)
        print(df)
        
    def cell(self, row, col):
        """
        Возвращает значения в ячейке
        rows-1 и col-1 -- чтобы адрес ячейки можно было задавать не 
        в питоновском формате, а в человеческом (значения начинаются с 1)
        """
        return self.table[row-1][col-1]
        
        
    def cell_change(self, row, col, value):
        self.table[row-1][col-1] = value
        
    def add_row(self, row_pos=-9999, row_values=[]):
        if row_pos == -9999:
            row_pos = len(self.table)
        if row_pos > (len(self.table)+1) or row_pos < 1 or isinstance(row_pos, int) == False:
            print('Row index out of range')
            return self.table
        if row_values == []:
            N = len(self.table[0])
            row_values = np.array([None] * N)
        if len(row_values) != len(self.table[0]):
            print('Недопустимая длина строки')
            return self.table
        row_pos = row_pos - 1
        self.table = np.insert(self.table, row_pos, row_values, axis=0)
        return self.table
    
    def add_col(self, col_pos=-9999, col_values=[]):
        if col_pos == -9999:
            col_pos = len(self.table[0])
        if col_pos > (len(self.table[0])+1) or col_pos < 1 or isinstance(col_pos, int) == False:
            print('Column index out of range')
            return self.table
        if col_values == []:
            N = len(self.table)
            col_values = np.array([None] * N)
        if len(col_values) != len(self.table):
            print('Недопустимая длина столбца')
            return self.table
        col_pos = col_pos - 1
        self.table = np.insert(self.table, col_pos, col_values, axis=1)
        return self.table
    
    def delete_row(self, row_pos):
        if row_pos > len(self.table) or row_pos < 1 or isinstance(row_pos, int) == False:
            print('Row index out of range')
            return self.table
        else:
            row_pos = row_pos - 1
            self.table = np.delete(self.table, row_pos, axis=0)
            return self.table
        
    def delete_col(self, col_pos):
        if col_pos > len(self.table[0]) or col_pos < 1 or isinstance(col_pos, int) == False:
            print('Column index out of range')
            return self.table
        else:
            col_pos = col_pos - 1
            self.table = np.delete(self.table, col_pos, axis=1)
            return self.table
    
    def set_range(self, row1, col1, row2, col2):
        """
        Возвращает массив значений из заданного диапазона
        """
        cells = []
        if row1 == row2 & col1 == col2:
            cells = self.table[row1-1][col1-1]
            print('В диапазоне только 1 ячейка')
        elif row1 == row2:
            a = min(col1,col2)
            b = max(col1,col2)
            for i in range (a-1,b):
                cells.append(self.table[row1-1][i])
        elif col1 == col2:
            a = min(row1,row2)
            b = max(row1,row2)
            for i in range (a-1,b):
                cells.append(self.table[i][col1-1])
        else:
            print('Неверно указан диапазон')
        return cells
    
    def range_operation(self, row1, col1, row2, col2, operation='sum'):
        """
        Операции над диапазоном. 
        """
        res = 0
        cells = self.set_range(row1 = row1, col1 = col1, row2 = row2,col2 = col2)
        if operation == 'sum':
            try:
                res = sum(cells)
                return res
            except:
                print('Расчет невозможен')
        if operation == 'mult':
            try:
                res = np.prod(cells)
                return res
            except:
                print('Расчет невозможен')
        if operation == 'mean':
            try:
                res = np.mean(cells)
                return res
            except:
                print('Расчет невозможен')
    
    def cell_operation(self,row1, col1, row2, col2, operation='add'):
        """
        Операции над ячейками
        """
        c1 = self.cell(row1, col1)
        c2 = self.cell(row2, col2)
        if operation == 'replace':
            self.table[row1-1][col1-1] = c2
            self.table[row2-1][col2-1] = c1
        else:
            if c1 is None or c2 is None:
                print('Операция над пустыми ячейками невозможна')
            elif type(c1) != type(c2):
                print('Значения в ячейках должны быть одного типа (str или int)')
            elif type(c1) == str:
                if operation == 'add':
                    res = c1 + c2
                    return res
                if operation == 'min':
                    if len(c1) <= len(c2):
                        res = c1
                    else:
                        res = c2
                    return res
                if operation == 'max':
                    if len(c1) >= len(c2):
                        res = c1
                    else:
                        res = c2
                    return res
            elif isinstance(c1, numbers.Number) == True:
                if operation == 'add':
                    res = c1 + c2
                    return res
                if operation == 'min':
                    if c1 <= c2:
                        res = c1
                    else:
                        res = c2
                    return res
                if operation == 'max':
                    if c1 >= c2:
                        res = c1
                    else:
                        res = c2
                    return res
            else:
                print('Не удалось произвести расчет')
            

Примеры использования

In [140]:
tb = SimpleTable('test.csv')
tb.print_table()

   1     2  3   4
1  a     b  3   d
2  1  None  2   3
3  2   sff  3  sd
4  1     2  3   4


In [143]:
#Изменение значения ячеек
print(tb.cell(3,2))
tb.cell_change(3, 2, 'ffs')
print(tb.cell(3,2))

sff
ffs


In [146]:
#Операции над диапазоном
print(tb.set_range(1,3,4,3))
print(tb.range_operation(1,3,4,3, operation='mean'))
print(tb.range_operation(1,3,4,3, operation='mult'))
print(tb.set_range(1,2,4,2))
print(tb.range_operation(1,2,4,2, operation='mean'))

[3, 2, 3, 3]
2.75
54
['b', None, 'ffs', 2]
Расчет невозможен
None


In [150]:
#Операции над ячейками
tb.print_table()
tb.cell_operation(1,1,4,4,operation='replace')
tb.print_table()

   1     2  3   4
1  a     b  3   d
2  1  None  2   3
3  2   ffs  3  sd
4  1     2  3   4
   1     2  3   4
1  4     b  3   d
2  1  None  2   3
3  2   ffs  3  sd
4  1     2  3   a


In [151]:
#Изменение размера таблицы
tb.add_col(col_pos=3, col_values = [11,12,13,14])
tb.print_table()
tb.add_col(col_pos=99, col_values = [11,12,13,14])
tb.print_table()

   1     2   3  4   5
1  4     b  11  3   d
2  1  None  12  2   3
3  2   ffs  13  3  sd
4  1     2  14  3   a
Column index out of range
   1     2   3  4   5
1  4     b  11  3   d
2  1  None  12  2   3
3  2   ffs  13  3  sd
4  1     2  14  3   a


In [152]:
tb.delete_col(3)
tb.print_table()

   1     2  3   4
1  4     b  3   d
2  1  None  2   3
3  2   ffs  3  sd
4  1     2  3   a
